In [68]:
import os
import pickle

In [69]:
import streamlit as st
from dotenv import load_dotenv

In [70]:
import pandas as pd

In [71]:
from utils.b2 import B2
from utils.modeling import *

In [72]:
# ------------------------------------------------------
#                      APP CONSTANTS
# ------------------------------------------------------
REMOTE_DATA1 = 'pbp-2013.csv'
REMOTE_DATA2 = 'pbp-2014.csv'
REMOTE_DATA3 = 'pbp-2015.csv'
REMOTE_DATA4 = 'pbp-2016.csv'

In [73]:
# ------------------------------------------------------
#                        CONFIG
# ------------------------------------------------------
load_dotenv()

True

In [74]:
# load Backblaze connection
b2 = B2(endpoint=os.environ['B2_ENDPOINT'],
        key_id=os.environ['B2_KEYID'],
        secret_key=os.environ['B2_APPKEY'])

In [75]:
# ------------------------------------------------------
#                        CACHING
# ------------------------------------------------------
@st.cache_data
def get_data(data):
    # collect data frame of reviews and their sentiment
    b2.set_bucket(os.environ['B2_BUCKETNAME'])
    df_pbp = b2.get_df(data)

    return df_pbp

2024-04-08 18:30:29.069 No runtime found, using MemoryCacheStorageManager


In [76]:
#data1 = get_data(REMOTE_DATA1)
data2 = get_data(REMOTE_DATA2)
data3 = get_data(REMOTE_DATA3)
data4 = get_data(REMOTE_DATA4)

In [77]:
data = pd.concat([data2, data3, data4], axis=0)

In [78]:
# data = get_data()
# data

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137923 entries, 0 to 45950
Data columns (total 45 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   GameId                          137923 non-null  int64  
 1   GameDate                        137923 non-null  object 
 2   Quarter                         137923 non-null  int64  
 3   Minute                          137923 non-null  int64  
 4   Second                          137923 non-null  int64  
 5   OffenseTeam                     128067 non-null  object 
 6   DefenseTeam                     137923 non-null  object 
 7   Down                            137923 non-null  int64  
 8   ToGo                            137923 non-null  int64  
 9   YardLine                        137923 non-null  int64  
 10  Unnamed: 10                     0 non-null       float64
 11  SeriesFirstDown                 137923 non-null  int64  
 12  Unnamed: 12           

In [80]:
# Remove incompletions to make the data looks a bit more normal
filtered_data = data[data['IsIncomplete'] != 1]
# Keep only rows where 'PlayType' is either 'PASS' or 'RUSH'
filtered_data = filtered_data[(filtered_data['PlayType'] == 'PASS') | (filtered_data['PlayType'] == 'RUSH')]

In [81]:
filtered_data.reset_index(drop=True, inplace=True)

In [82]:
# Create a dictionary to map unique offense team names to unique identifiers
team_to_teamid = {team: i for i, team in enumerate(filtered_data['OffenseTeam'].unique(), start=1)}
# Create the 'TeamID' column by mapping the 'OffenseTeam' column to the unique identifiers
filtered_data['TeamID'] = filtered_data['OffenseTeam'].map(team_to_teamid)
# Creating a unique situation identifier
filtered_data['SitID'] = filtered_data['YardLine'] + 100 * filtered_data['ToGo'] + 10000 * filtered_data['Down'] + 100000 * filtered_data['TeamID']

In [83]:
#csv_file_path = 'filtered_data.csv'
#filtered_data.to_csv(csv_file_path, index=False)

In [84]:
pass_type_mapping = {pass_type: i for i, pass_type in enumerate(filtered_data['PassType'].unique(), start=1)}
rush_direction_mapping = {rush_direction: i for i, rush_direction in enumerate(filtered_data['RushDirection'].unique(), start=1)}

In [85]:
# Create the 'PlayID' column
filtered_data['PlayID'] = filtered_data['IsRush'] * 100 + filtered_data['RushDirection'].map(rush_direction_mapping)
filtered_data.loc[filtered_data['IsPass'] == 1, 'PlayID'] = 200 + filtered_data['PassType'].map(pass_type_mapping)

In [86]:
filtered_data.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,TeamID,SitID,PlayID
0,2014113007,2014-11-30,4,2,55,LA,LV,2,3,50,...,50,OPP,0,NaN,0,NaN,0,1,120350,101
1,2014113007,2014-11-30,4,3,39,LA,LV,1,10,43,...,43,OWN,0,NaN,0,NaN,0,1,111043,101
2,2014113007,2014-11-30,4,8,8,LA,LV,3,11,35,...,35,OWN,0,NaN,0,NaN,0,1,131135,101
3,2014113007,2014-11-30,4,8,50,LA,LV,2,10,36,...,36,OWN,0,NaN,0,NaN,0,1,121036,102
4,2014113007,2014-11-30,4,9,30,LA,LV,1,10,36,...,36,OWN,0,NaN,0,NaN,0,1,111036,102


In [87]:
# Specify the file path for the CSV file
csv_file_path = 'pbp.csv'
# Export the DataFrame to a CSV file, overriding the existing file if it exists
filtered_data.to_csv(csv_file_path, index=False, mode='w')